In [2]:
import os
import glob
import pandas as pd
import numpy as np
import numpy.matlib
import scipy as sc
from sklearn.model_selection import KFold
import warnings
warnings.filterwarnings('ignore')
pd.set_option('max_columns', 300)
import pickle
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler, QuantileTransformer
import sys
sys.path.append('..')

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
import torch.nn.functional as F
from torch.utils.data import DataLoader

from utils import create_save_folder, EarlyStopping

os.environ["CUDA_VISIBLE_DEVICES"]="1"
fm_path = create_save_folder('nb025')

In [3]:
out_train = pd.read_csv('../input/optiver-realized-volatility-prediction/train.csv')
out_train = out_train.pivot(index='time_id', columns='stock_id', values='target')

out_train = out_train.fillna(out_train.mean())
out_train.head()

stock_id,0,1,2,3,4,5,6,7,8,9,10,11,13,14,15,16,17,18,19,20,21,22,23,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,46,47,48,50,51,52,53,55,56,58,59,60,61,62,63,64,66,67,68,69,70,72,73,74,75,76,77,78,80,81,82,83,84,85,86,87,88,89,90,93,94,95,96,97,98,99,100,101,102,103,104,105,107,108,109,110,111,112,113,114,115,116,118,119,120,122,123,124,125,126
time_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
5,0.004136,0.006340,0.001848,0.005300,0.004468,0.006234,0.007651,0.003624,0.010036,0.007291,0.005707,0.005918,0.002148,0.002627,0.005091,0.005485,0.004615,0.016663,0.002883,0.002915,0.002218,0.006377,0.006338,0.004405,0.008170,0.003655,0.002963,0.003233,0.004113,0.002649,0.006633,0.002586,0.003092,0.004755,0.008078,0.008372,0.002707,0.004864,0.002184,0.004094,0.002090,0.004551,0.001848,0.002206,0.003171,0.003947,0.002345,0.004885,0.002594,0.007041,0.005736,0.003274,0.004072,0.003928,0.004460,0.008681,0.004841,0.001808,0.004759,0.002675,0.002242,0.002625,0.002841,0.005403,0.005920,0.002803,0.005253,0.003856,0.002477,0.006344,0.007048,0.005846,0.003187,0.006131,0.005706,0.003294,0.003858,0.006019,0.004970,0.003590,0.008780,0.002126,0.005341,0.004152,0.005130,0.006254,0.003832,0.001742,0.004580,0.006088,0.005514,0.005007,0.003218,0.002995,0.004273,0.003129,0.004710,0.007743,0.002893,0.005967,0.006204,0.003708,0.004267,0.007944,0.003336,0.002571,0.003035,0.004862,0.002942,0.004112,0.001919,0.008067
11,0.001445,0.002099,0.000806,0.002774,0.001852,0.002562,0.004670,0.002458,0.002291,0.002529,0.002352,0.001634,0.000640,0.001072,0.001826,0.003172,0.002474,0.003902,0.003411,0.001601,0.001622,0.002445,0.002464,0.001417,0.003665,0.001991,0.000663,0.001917,0.000956,0.000688,0.007245,0.000924,0.001146,0.001555,0.004299,0.002344,0.001121,0.004665,0.000652,0.001330,0.000472,0.001861,0.000728,0.000760,0.001998,0.001296,0.001610,0.001958,0.001335,0.003729,0.002891,0.001223,0.001489,0.001607,0.001718,0.003743,0.001663,0.001230,0.001405,0.001449,0.001769,0.001133,0.002418,0.001837,0.002326,0.002262,0.003308,0.002284,0.001251,0.003299,0.004387,0.003544,0.001672,0.001918,0.002037,0.001285,0.001493,0.002496,0.001209,0.001609,0.004821,0.001003,0.002341,0.001560,0.002393,0.002656,0.001619,0.001013,0.002152,0.002691,0.001486,0.001788,0.001224,0.000965,0.001204,0.001339,0.002119,0.002275,0.001195,0.003635,0.001859,0.002443,0.001382,0.002469,0.002030,0.000839,0.001271,0.002095,0.001518,0.001891,0.001123,0.003965
16,0.002168,0.002456,0.001581,0.002986,0.002213,0.003253,0.004303,0.002178,0.001841,0.003299,0.002363,0.003923,0.001702,0.002114,0.002539,0.002653,0.002831,0.003806,0.002137,0.001902,0.001629,0.002742,0.002465,0.002162,0.006047,0.002152,0.001239,0.002344,0.002127,0.002431,0.003436,0.001612,0.002205,0.001976,0.004228,0.002748,0.001477,0.004131,0.001774,0.002267,0.001275,0.002703,0.001308,0.001800,0.002026,0.002777,0.001783,0.002092,0.001784,0.003094,0.002886,0.003132,0.002563,0.002423,0.002020,0.003401,0.001461,0.001461,0.002030,0.002063,0.002289,0.001413,0.002510,0.001670,0.002804,0.003364,0.003708,0.002256,0.002166,0.002735,0.002815,0.003735,0.002981,0.001996,0.002094,0.002147,0.001559,0.002654,0.002217,0.002112,0.002367,0.001688,0.001896,0.002378,0.003085,0.003712,0.003618,0.001366,0.002965,0.002639,0.003431,0.002939,0.002458,0.001799,0.001518,0.002630,0.002738,0.002752,0.001753,0.003406,0.002687,0.002704,0.001949,0.002195,0.003410,0.002569,0.002137,0.001893,0.002131,0.002428,0.001548,0.003161
31,0.002195,0.002807,0.001599,0.004437,0.002256,0.003072,0.005401,0.002149,0.003997,0.003696,0.002341,0.003581,0.001335,0.001802,0.002007,0.001897,0.002201,0.005206,0.001827,0.002071,0.001471,0.002550,0.002127,0.001638,0.004563,0.001634,0.000883,0.001073,0.003748,0.001881,0.002581,0.001369,0.002386,0.002128,0.002666,0.002927,0.001557,0.001945,0.000841,0.001757,0.000410,0.002066,0.001285,0.001337,0.001742,0.002524,0.001936,0.002696,0.001466,0.004116,0.003129,0.002059,0.002323,0.002589,

knn++を使ってkfoldの分割を行う

In [4]:
nfolds = 5
index = []
totDist = []
values = []

mat = out_train.values

scaler = MinMaxScaler(feature_range=(-1, 1))
mat = scaler.fit_transform(mat)

nind = int(mat.shape[0] / nfolds)

mat = np.c_[mat, np.arange(mat.shape[0])]

lineNumber = np.random.choice(np.array(mat.shape[0]), size=nfolds, replace=False)
lineNumber = np.sort(lineNumber)[::-1]
lineNumber

array([3741, 3262, 2191,  911,  380])

たぶんKNN++の実装

In [5]:
for n in range(nfolds):
    totDist.append(np.zeros(mat.shape[0]-nfolds))

# saves index
for n in range(nfolds):
    
    values.append([lineNumber[n]])    


s=[]
for n in range(nfolds):
    s.append(mat[lineNumber[n],:])
    
    mat = np.delete(mat, obj=lineNumber[n], axis=0)

for n in range(nind-1):    

    luck = np.random.uniform(0,1,nfolds)
    
    for cycle in range(nfolds):
        s[cycle] = np.matlib.repmat(s[cycle], mat.shape[0], 1)

        sumDist = np.sum( (mat[:,:-1] - s[cycle][:,:-1])**2 , axis=1)   
        totDist[cycle] += sumDist        
                
        # probabilities
        f = totDist[cycle]/np.sum(totDist[cycle]) # normalizing the totdist
        j = 0
        kn = 0
        for val in f:
            j += val        
            if (j > luck[cycle]): # the column was selected
                break
            kn +=1
        lineNumber[cycle] = kn
        
        # delete line of the value added    
        for n_iter in range(nfolds):
            
            totDist[n_iter] = np.delete(totDist[n_iter],obj=lineNumber[cycle], axis=0)
            j= 0
        
        s[cycle] = mat[lineNumber[cycle],:]
        values[cycle].append(int(mat[lineNumber[cycle],-1]))
        mat = np.delete(mat, obj=lineNumber[cycle], axis=0)


for n_mod in range(nfolds):
    values[n_mod] = out_train.index[values[n_mod]]

In [6]:
with open('/home/yoshikawa/work/kaggle/OPVP/output/feature_model/20210920/lgbm_group_kfold_time_id/train.pkl', 'rb') as f:
    df_train = pickle.load(f)

df_train['time_id'] = df_train['row_id'].apply(lambda x:x.split('-')[1])

col_names = list(df_train)
col_names.remove('target')
col_names.remove('row_id')
col_names.remove('stock_id')
col_names.remove('time_id')
len(col_names)

1020

In [7]:
df_train.replace([np.inf, -np.inf], np.nan,inplace=True)
# qt_train = []

for col in col_names:
    #正規分布になるよう非線形変換
    qt = QuantileTransformer(random_state=55, n_quantiles=2000, output_distribution='normal')
    df_train[col] = qt.fit_transform(df_train[[col]])
    # qt_train.append(qt)


In [8]:
n_folds =5
kf = KFold(n_splits=n_folds, shuffle=True, random_state=55)
scores = []

train_columns = list(df_train)
train_columns.remove('time_id')
train_columns.remove('row_id')
train_columns.remove('target')

for col in train_columns:
    df_train[col] = df_train[col].fillna(df_train[col].mean())
df_train.describe()

,target,log_return_realized_volatility,log_return_mean,log_return_std,log_return_up_rate,log_return_last_value,log_return2_realized_volatility,log_return2_mean,log_return2_std,log_return2_up_rate,log_return2_last_value,log_return3_realized_volatility,log_return3_mean,log_return3_std,log_return3_up_rate,log_return3_last_value,log_return4_realized_volatility,log_return4_mean,log_return4_std,log_return4_up_rate,log_return4_last_value,wap_balance_mean,wap_balance_std,wap_balance_amax,wap_balance2_mean,wap_balance2_std,wap_balance2_amax,price_spread_mean,price_spread_std,price_spread_amax,price_spread2_mean,price_spread2_std,price_spread2_amax,bid_spread_mean,bid_spread_std,bid_spread_amax,ask_spread_mean,ask_spread_std,ask_spread_amax,volume_imbalance_mean,volume_imbalance_std,volume_imbalance_amax,total_volume_mean,total_volume_std,total_volume_amax,ask_volume_mean,ask_volume_std,ask_volume_amax,bid_volume_mean,bid_volume_std,bid_volume_amax,wap_mean,wap_std,wap_up_rate,wap_last_value,wap2_mean,wap2_std,wap2_up_rate,wap2_last_value,wap3_mean,wap3_std,wap3_up_rate,wap3_last_value,wap4_mean,wap4_std,wap4_up_rate,wap4_last_value,log_return_realized_volatility_500,log_return_mean_500,log_return_std_500,log_return_up_rate_500,log_return_last_value_500,log_return2_realized_volatility_500,log_return2_mean_500,log_return2_std_500,log_return2_up_rate_500,log_return2_last_value_500,log_return3_realized_volatility_500,log_return3_mean_500,log_return3_std_500,log_return3_up_rate_500,log_return3_last_value_500,log_return4_realized_volatility_500,log_return4_mean_500,log_return4_std_500,log_return4_up_rate_500,log_return4_last_value_500,wap_balance_mean_500,wap_balance_std_500,wap_balance_amax_500,wap_balance2_mean_500,wap_balance2_std_500,wap_balance2_amax_500,price_spread_mean_500,price_spread_std_500,price_spread_amax_500,price_spread2_mean_500,price_spread2_std_500,price_spread2_amax_500,bid_spread_mean_500,bid_spread_std_500,bid_spread_amax_500,ask_spread_mean_500,ask_spread_std_500,ask_spread_amax_500,volume_imbalance_mean_500,volume_imbalance_std_500,volume_imbalance_amax_500,total_volume_mean_500,total_volume_std_500,total_volume_amax_500,ask_volume_mean_500,ask_volume_std_500,ask_volume_amax_500,bid_volume_mean_500,bid_volume_std_500,bid_volume_amax_500,wap_mean_500,wap_std_500,wap_up_rate_500,wap_last_value_500,wap2_mean_500,wap2_std_500,wap2_up_rate_500,wap2_last_value_500,wap3_mean_500,wap3_std_500,wap3_up_rate_500,wap3_last_value_500,wap4_mean_500,wap4_std_500,wap4_up_rate_500,wap4_last_value_500,log_return_realized_volatility_400,log_return_mean_400,log_return_std_400,log_return_up_rate_400,log_return_last_value_400,log_return2_realized_volatility_400,log_return2_mean_400,log_return2_std_400,log_return2_up_rate_400,log_return2_last_value_400,log_return3_realized_volatility_400,log_return3_mean_400,log_return3_std_400,log_return3_up_rate_400,log_return3_last_value_400,log_return4_realized_volatility_400,log_return4_mean_400,...,trade_log_return_per_size_realized_volatility_min_time,trade_log_return_per_amount_realized_volatility_mean_time,trade_log_return_per_amount_realized_volatility_std_time,trade_log_return_per_amount_realized_volatility_max_time,trade_log_return_per_amount_realized_volatility_min_time,trade_log_return_seconds_diff_realized_volatility_mean_time,trade_log_return_seconds_diff_realized_volatility_std_time,trade_log_return_seconds_diff_realized_volatility_max_time,trade_log_return_seconds_diff_realized_volatility_min_time,trade_log_return_realized_volatility_500_mean_time,trade_log_return_realized_volatility_500_std_time,trade_log_return_realized_volatility_500_max_time,trade_log_return_realized_volatility_500_min_time,trade_log_return_per_size_realized_volatility_500_mean_time,trade_log_return_per_size_realized_volatility_500_std_time,trade_log_return_per_size_realized_volatility_500_max_time,trade_log_return_per_size_realized_volatility_500_min_time,trade_log_return_per_amount_realized_volatility_500_mean_time,t

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def ToTensor(x):
    return torch.tensor(x.astype(np.float32)).to(device)
from torch.utils.data import Dataset

class DataSet(Dataset):
    def __init__(self, X, stock, y):
        self.X = X
        self.stock = stock
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, index):
        return self.X[index], self.stock[index], self.y[index]

In [10]:
def rsmpe(y_true, y_pred):
    rsmpe = (torch.sqrt(torch.clamp(torch.mean(torch.square((y_true - y_pred) / y_true)),  min=1.0e-6)))
    assert rsmpe != 0
    return rsmpe

In [11]:
df_train.isnull().values.sum()

0

In [12]:
import subprocess
import shlex

def gpuinfo():
    """
    Returns size of total GPU RAM and used GPU RAM.

    Parameters
    ----------
    None

    Returns
    -------
    info : dict
        Total GPU RAM in integer for key 'total_MiB'.
        Used GPU RAM in integer for key 'used_MiB'.
    """

    command = 'nvidia-smi -q -d MEMORY | sed -n "/FB Memory Usage/,/Free/p" | sed -e "1d" -e "4d" -e "s/ MiB//g" | cut -d ":" -f 2 | cut -c2-'
    commands = [shlex.split(part) for part in command.split(' | ')]
    for i, cmd in enumerate(commands):
        if i==0:
            res = subprocess.Popen(cmd, stdout=subprocess.PIPE)
        else:
            res = subprocess.Popen(cmd, stdin=res.stdout, stdout=subprocess.PIPE)

    l = res.communicate()[0].decode('utf-8').strip().split('\n')
    info = {'total_MiB':l[3], 'used_MiB':l[4]}
    return info

In [13]:
gpuinfo()

{'total_MiB': '24256', 'used_MiB': '3'}

In [14]:
%load_ext autoreload
%autoreload
from model import FFNN_v2

models = []
t_losses = [[] for i in range(n_folds)]
v_losses = [[] for i in range(n_folds)]
for fold in range(n_folds):
    indexes = np.arange(n_folds).astype(int)    
    indexes = np.delete(indexes,obj=fold, axis=0) 
    indexes = np.r_[values[indexes[0]],values[indexes[1]],values[indexes[2]],values[indexes[3]]]
    indexes = [str(x) for x in indexes]
    values[fold] = [str(x) for x in values[fold]]

    X_train = df_train[df_train['time_id'].isin(indexes)]
    X_val = df_train.loc[df_train.time_id.isin(values[fold])]
    scaler = MinMaxScaler(feature_range=(-1, 1))

    num_data = X_train.drop(['stock_id', 'target', 'time_id', 'row_id'], axis=1)
    num_data = scaler.fit_transform(num_data)
    stock = X_train[['stock_id']].values
    y = X_train[['target']].values

    num_data_val = X_val.drop(['stock_id', 'target', 'time_id', 'row_id'], axis=1)
    num_data_val = scaler.transform(num_data_val) 
    stock_val = X_val[['stock_id']].values
    y_val = X_val[['target']].values
    
    train_dataset = DataSet(num_data, stock, y)
    train_loader = DataLoader(train_dataset, 4096, shuffle=True)
    
    model = FFNN_v2(num_data.shape[1]).cuda()
    print(model)
    optimizer = optim.Adam(model.parameters(), lr=0.005)
    earlystopping = EarlyStopping(patience=20, verbose=True, path=fm_path+'/checkpoint.pth')
    scheduler = ReduceLROnPlateau(optimizer, 'min', factor=0.2, patience=7)
    
    epochs = 1000
    model.train()
    for i in range(epochs):
        train_loss, val_loss = 0, 0
        for j, (X, stock, y)in enumerate(train_loader):
            X, stock, y = X.to(torch.float).cuda(), stock.cuda(), y.to(torch.float).cuda()
            out = model(X, stock)
            loss = rsmpe(y, out)
            # print(loss)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            train_loss += loss.item() * X.shape[0]
        
        train_loss /= len(train_dataset)
        out = model(ToTensor(num_data_val), torch.tensor(stock_val).cuda())
        val_loss = rsmpe(ToTensor(y_val), out).item()
        
        print(f"{i+1} epoch - train_loss: {train_loss:.6f} val_loss: {val_loss:.6f}")
        # print(gpuinfo())
        t_losses[fold].append(train_loss)
        v_losses[fold].append(val_loss)
        earlystopping(val_loss, model)
        scheduler.step(val_loss)
        if earlystopping.early_stop:
            print("Early Stopping!!")
            break
    model.load_state_dict(torch.load(fm_path+'/checkpoint.pth'))
    models.append(model)


FFNN_v2(
  (stock_embedding): Embedding(127, 24)
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (hidden): Sequential(
    (0): Linear(in_features=1044, out_features=256, bias=True)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): SiLU(inplace=True)
    (3): Linear(in_features=256, out_features=128, bias=True)
    (4): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): SiLU(inplace=True)
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): SiLU(inplace=True)
    (9): Linear(in_features=64, out_features=32, bias=True)
    (10): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): SiLU(inplace=True)
  )
  (out): Linear(in_features=32, out_features=1, bias=True)
)
1 epoch - train_loss: 18.922561 val_loss: 4.530925
Validation loss decreased (inf --> 4.530925).  

In [15]:
for i, model in enumerate(models):
	torch.save(model.cpu().state_dict(), fm_path+'/FFNNmodel-'+str(i))


In [16]:
# import matplotlib.pyplot as plt

# for fold in range(n_folds):
# 	x = range(len(t_losses[fold]))
# 	x = [(i+1) * 5 for i in x]
# 	print(x)

In [17]:
X_train.drop(['stock_id', 'target', 'time_id', 'row_id'], axis=1).columns.to_series().to_csv(os.path.join(fm_path, "columns.csv"), index=False)
